# Mutation-Based Fuzzing

## Fuzzing a URL Parser

In [4]:
from ipynb.fs.full.fuzzer import *

In [5]:
fuzzer()

'<0&=10%:59,=;";1($>%?)\')084<4,9*8.66 "%.%$=: 75,!,$0:>",\'<1("550?+.?/#5,*+4"#-,9525=4#'

In [1]:
from typing import Tuple, List, Callable, Any, Set

In [11]:
from urllib.parse import urlparse

In [3]:
urlparse("http://www.google.com/search?q=fuzzing")

ParseResult(scheme='http', netloc='www.google.com', path='/search', params='', query='q=fuzzing', fragment='')

In [8]:
def http_program(url: str) -> bool:
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + 
                         repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    # Do something with the URL
    return True

In [9]:
fuzzer(char_start=32, char_range=96)

"|oZQ2\\=uW'J#Z"

In [15]:
for i in range(10000):
    try:
        url = fuzzer()
        result = http_program(url)
        print("Success!")
    except ValueError:
        pass

## Mutating Inputs

In [16]:
import random

In [17]:
def delete_random_character(s: str) -> str:
    if s == '':
        return s
    pos = random.randrange(0, len(s))
    return s[:pos] + s[pos + 1:]

In [18]:
seed_input = "A quick brown fox"
for i in range(10):
    x = delete_random_character(seed_input)
    print(repr(x))

'A uick brown fox'
'A quck brown fox'
'A quik brown fox'
'A qick brown fox'
'A uick brown fox'
'A quick brow fox'
'A quck brown fox'
'A quik brown fox'
'A quick bron fox'
'A quic brown fox'


In [19]:
def insert_random_character(s: str) -> str:
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]

In [20]:
for i in range(10):
    print(repr(insert_random_character(seed_input)))

'A qucick brown fox'
'A quick brown: fox'
'A quick brown f@ox'
'A Kquick brown fox'
'A Zquick brown fox'
'A quick br&own fox'
'A quick browwn fox'
'A quick brown foxs'
'A` quick brown fox'
'A quick brow}n fox'


In [21]:
2^1

3

In [22]:
2^2

0

In [23]:
2 << 3

16

In [24]:
2 >> 1

1

In [25]:
def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    # print("Flipping", bit, "in", repr(c) + ", giving", repr(new_c))
    return s[:pos] + new_c + s[pos + 1:]

In [26]:
for i in range(10):
    print(repr(flip_random_character(seed_input)))

'A quisk brown fox'
'A quick bvown fox'
'A quick brown fmx'
'A(quick brown fox'
'A quick brown box'
'A quick brmwn fox'
'C quick brown fox'
'A quick brosn fox'
'A qu)ck brown fox'
'A quick brown foy'


In [27]:
def mutate(s: str) -> str:
    """Return s with a random mutation applied"""
    mutators = [
        delete_random_character,
        insert_random_character,
        flip_random_character
    ]
    mutator = random.choice(mutators)
    # print(mutator)
    return mutator(s)

In [28]:
for i in range(10):
    print(repr(mutate("A quick brown fox")))

'A quick b2own fox'
' quick brown fox'
'A uick brown fox'
'A quick0brown fox'
'A quick brown foxS'
'A quick bbown fox'
'A quick brow. fox'
'A quikk brown fox'
'A`quick brown fox'
'A quick crown fox'


## Mutating URLS

In [29]:
def is_valid_url(url: str) -> bool:
    try:
        result = http_program(url)
        return True
    except ValueError:
        return False

In [30]:
assert is_valid_url("http://www.google.com/search?q=fuzzing")
assert not is_valid_url("xyzzy")

In [31]:
seed_input = "http://www.google.com/search?q=fuzzing"
valid_inputs = set()
trials = 20

for i in range(trials):
    inp = mutate(seed_input)
    if is_valid_url(inp):
        valid_inputs.add(inp)

In [32]:
len(valid_inputs) / trials

0.7

In [33]:
valid_inputs

{'http://ww.google.com/search?q=fuzzing',
 'http://www.gogle.com/search?q=fuzzing',
 'http://www.googe.com/search?q=fuzzing',
 "http://www.googl'e.com/search?q=fuzzing",
 'http://www.google.:com/search?q=fuzzing',
 'http://www.google.com/sAearch?q=fuzzing',
 'http://www.google.com/search"?q=fuzzing',
 'http://www.google.com/search?q=Wfuzzing',
 'http://www.google.com/search?q=fuzing',
 'http://www.google.com/search?q=fuzzig',
 'http://www.google.com/search?q@=fuzzing',
 'http://www.google.com/searh?q=fuzzing',
 'http://www.google.com/serarch?q=fuzzing',
 'http://www.goole.com/search?q=fuzzing'}

## MUltiple Mutations

In [34]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutations = 50

In [35]:
inp = seed_input
for i in range(mutations):
    if i % 5 == 0:
        print(i, "mutations:", repr(inp))
    inp = mutate(inp)

0 mutations: 'http://www.google.com/search?q=fuzzing'
5 mutations: 'http//www.google.kom/serch?y=fu2zzing'
10 mutations: 'http//www.google.kom/serch?=vuzang'
15 mutations: '(tt,p//www.googcle.kom/&serch?=v3uzang'
20 mutations: '(tt,0/www.googc:le.kom/&serch?=v3qzan'
25 mutations: '(tt,0/wcwwL.googc:le.kom/&rrc?=v3qzan'
30 mutations: '(tt,0/cwwL.googc:l.kom+&r2c?=v3qxan'
35 mutations: '(tt,0/cwwL.gogc:l.kom+&r2b?=v3FxAn'
40 mutations: '(tt,0/cwwL.gogdc:l.kom)+&r2b?k=3FxAn'
45 mutations: '(tt,00/cwwL.gogd:l.kom)x+&r2b?k=3IxAn'


In [36]:
Fuzzer

ipynb.fs.full.fuzzer.Fuzzer

In [37]:
class MutationFuzzer(Fuzzer):
    """Base class for mutational fuzzing"""

    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        """Constructor.
        `seed` - a list of (input) strings to mutate.
        `min_mutations` - the minimum number of mutations to apply.
        `max_mutations` - the maximum number of mutations to apply.
        """
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        """Set population to initial seed.
        To be overloaded in subclasses."""
        self.population = self.seed
        self.seed_index = 0

In [38]:
class MutationFuzzer(MutationFuzzer):
    def mutate(self, inp: str) -> str:
        return mutate(inp)

## Guiding by Coverage